This notebook demonstrates the **new policy-based** behavior first (no use of `predict_reject`).
At the end we show the legacy `predict_reject` workflow with a clear note.

In [49]:
from __future__ import annotations
import sys
from pathlib import Path
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# ensure local src on path
repo_root = Path.cwd()
if not (repo_root / "src" / "calibrated_explanations").exists():
    if (repo_root.parent / "src" / "calibrated_explanations").exists():
        repo_root = repo_root.parent
src_path = str(repo_root / "src")
if src_path not in sys.path:
    sys.path.insert(0, src_path)
import calibrated_explanations as ce_pkg
from calibrated_explanations import CalibratedExplainer

print("Imported from:", Path(ce_pkg.__file__).resolve())

Imported from: C:\Users\loftuw\Documents\Github\kristinebergs-calibrated_explanations\src\calibrated_explanations\__init__.py


In [50]:
# Data and model
X, y = load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_proper, X_cal, y_proper, y_cal = train_test_split(
    X_train, y_train, test_size=0.25, random_state=42
)
model = RandomForestClassifier(n_estimators=10, random_state=42)
model.fit(X_proper, y_proper)
ce = CalibratedExplainer(model, X_cal, y_cal)
ce.initialize_reject_learner()

ConformalClassifier(fitted=True, mondrian=True)

In [51]:
# Helper summarizer tuned for explain_factual outputs
def _summarize_result(result, *, label: str):
    is_reject_result = hasattr(result, "policy") and hasattr(result, "rejected")
    print("\n---", label, "---")
    print("type:", type(result))
    if not is_reject_result:
        if hasattr(result, "__len__") and not isinstance(result, (str, bytes, dict)):
            try:
                print("len:", len(result))
                # If this is a CalibratedExplanations collection, count non-None explanations
                expls = getattr(result, "explanations", None)
                if expls is not None:
                    total = len(expls)
                    explained = sum(1 for e in expls if e is not None)
                    print(f"explained_count: {explained} / {total}")
                else:
                    try:
                        explained = sum(1 for e in result if e is not None)
                        print(f"explained_count (iter): {explained} / {len(result)}")
                    except Exception:
                        pass
            except Exception:
                pass
        else:
            print(repr(result))
        return
    print("policy:", result.policy)
    rejected = getattr(result, "rejected", None)
    if rejected is not None:
        rejected = np.asarray(rejected, dtype=bool)
        print("rejected_count:", int(rejected.sum()), "/", int(rejected.size))
    pred = getattr(result, "prediction", None)
    print("has_prediction:", pred is not None)
    expl = getattr(result, "explanation", None)
    if expl is None:
        print("explanation: None")
    else:
        if hasattr(expl, "__len__") and not isinstance(expl, (str, bytes, dict)):
            print("explanations_len:", len(expl))
            for i, e in enumerate(expl):
                if i >= 5:
                    print("... (truncated)")
                    break
                if e is None:
                    print(f" [{i}] None")
                    continue
                if hasattr(e, "prediction"):
                    try:
                        p = e.prediction
                        if isinstance(p, dict) and "predict" in p:
                            print(f' [{i}] predict={p["predict"]:.3f}')
                        else:
                            print(f" [{i}] prediction_present")
                    except Exception:
                        print(f" [{i}] prediction (unprintable)")
                else:
                    print(f" [{i}] explanation_type={type(e)}")
        else:
            print("explanation_type:", type(expl))

In [52]:
from calibrated_explanations.core.reject.policy import RejectPolicy

POLICIES = [
    RejectPolicy.NONE,
    RejectPolicy.PREDICT_AND_FLAG,
    RejectPolicy.EXPLAIN_ALL,
    RejectPolicy.EXPLAIN_REJECTS,
    RejectPolicy.EXPLAIN_NON_REJECTS,
    RejectPolicy.SKIP_ON_REJECT,
]
# Policy-first demo: show predict_internal outputs for each policy
for policy in POLICIES:
    res = ce.predict_internal(X_test[:4], reject_policy=policy)
    _summarize_result(res, label=f"predict_internal(reject_policy={policy.value})")

# Then show explain_factual with each policy (deterministic plugin off)
for policy in POLICIES:
    res = ce.explain_factual(X_test[:4], reject_policy=policy, _use_plugin=False)
    _summarize_result(res, label=f"explain_factual(reject_policy={policy.value})")

# Show explainer-level default policy behaviour
ce.default_reject_policy = RejectPolicy.EXPLAIN_REJECTS
res_def = ce.explain_factual(X_test[:4], _use_plugin=False)
_summarize_result(res_def, label="explain_factual(default_reject_policy=EXPLAIN_REJECTS)")
ce.default_reject_policy = RejectPolicy.NONE


--- predict_internal(reject_policy=none) ---
type: <class 'tuple'>
len: 4
explained_count (iter): 3 / 4

--- predict_internal(reject_policy=predict_and_flag) ---
type: <class 'calibrated_explanations.explanations.reject.RejectResult'>
policy: RejectPolicy.PREDICT_AND_FLAG
rejected_count: 0 / 4
has_prediction: True
explanation: None

--- predict_internal(reject_policy=explain_all) ---
type: <class 'calibrated_explanations.explanations.reject.RejectResult'>
policy: RejectPolicy.EXPLAIN_ALL
rejected_count: 1 / 4
has_prediction: True
explanations_len: 4
 [0] explanation_type=<class 'numpy.ndarray'>
 [1] explanation_type=<class 'numpy.ndarray'>
 [2] explanation_type=<class 'numpy.ndarray'>
 [3] None

--- predict_internal(reject_policy=explain_rejects) ---
type: <class 'calibrated_explanations.explanations.reject.RejectResult'>
policy: RejectPolicy.EXPLAIN_REJECTS
rejected_count: 0 / 4
has_prediction: True
explanation: None

--- predict_internal(reject_policy=explain_non_rejects) ---
type: 

## Legacy: `predict_reject`-based triage (legacy behaviour)
The examples below show the older workflow that explicitly calls `predict_reject` and then performs manual triage/explanations.
This legacy path is fully supported but the policy-first examples above are the recommended pattern going forward.

In [53]:
# Legacy example using predict_reject
rejected_mask, error_rate, reject_rate = ce.predict_reject(X_test, confidence=0.95)
rejected_mask = np.asarray(rejected_mask, dtype=bool)
print("Rejected (first 10):", rejected_mask[:10].tolist())
print(f"reject_rate={reject_rate:.3f}, error_rate={error_rate:.3f}")

# Triage loop (legacy)
for i in range(min(10, len(y_test))):
    if rejected_mask[i]:
        print(f"Instance {i}: Rejected - requires human review", end="")
        explanations = ce.explain_factual(X_test[i : i + 1], _use_plugin=False)
        print(f'  Probability: {explanations[0].prediction["predict"]:.3f}')
    else:
        print(f"Instance {i}: Auto-decided - confident prediction", end="")
        pred = ce.predict_proba(X_test[i : i + 1])
        print(f"  Probability: {pred[0][1]:.3f}")

Rejected (first 10): [True, False, False, False, False, False, False, False, False, True]
reject_rate=0.105, error_rate=0.056
Instance 0: Rejected - requires human review  Probability: 0.860
Instance 1: Auto-decided - confident prediction  Probability: 0.026
Instance 2: Auto-decided - confident prediction  Probability: 0.026
Instance 3: Auto-decided - confident prediction  Probability: 0.960
Instance 4: Auto-decided - confident prediction  Probability: 0.960
Instance 5: Auto-decided - confident prediction  Probability: 0.026
Instance 6: Auto-decided - confident prediction  Probability: 0.026
Instance 7: Auto-decided - confident prediction  Probability: 0.125
Instance 8: Auto-decided - confident prediction  Probability: 0.026
Instance 9: Rejected - requires human review  Probability: 0.847
